In [6]:
def daryk(x,y):
    a = x + y
    return (a)

daryk(1,2)

3

In [7]:
class Daiktas:

    def __init__(self, x, y):
        self.x = x
        self.y = y

daiktas1 = Daiktas(x=1, y=2)

daiktas1.x

daiktas1.y

d2 = Daiktas(x=10, y=30)

print (d2)

In [1]:
import glob
import imageio
import pandas as pd
import matplotlib.pyplot as plt

class Pasaulis:

    def __init__(self):
        self.laikas = 0   # [s]
        self.objektu_sarasas = []

    def vaizduoja(self, step):
        fig, (ax1, ax2) = plt.subplots(
            1, 2,
            figsize=(9, 4)   # coliai / inches
        )

        for objektas in self.objektu_sarasas:
            ax1.scatter(
                objektas.x,
                objektas.y,
            )

        ax1.set_xlim(0, 10)
        ax1.set_ylim(0, 10)
        ax1.set_xlabel('$x$ [m]')
        ax1.set_ylabel('$y$ [m]')
        ax1.tick_params(
            direction='in',
            top=True,
            right=True
        )

        fig.savefig(f'{step:03d}.png', dpi=100)
        plt.clf()
        del fig


    def sukasi(self, delta_t, n_steps):
        # delta_t [s]
        for step in range(n_steps):
            self.laikas += delta_t
            for objektas in self.objektu_sarasas:
                objektas.juda(self.laikas, delta_t)   # TODO patikrinti logika
            self.vaizduoja(step)


class Objektas:

    def __init__(self,
                 x_start, y_start,
                 vx_start, vy_start):

        self.logeris = {
            'laikas': [0],   # TODO laikas 0
            'x': [x_start],
            'y': [y_start],
            'vx': [vx_start],
            'vy': [vy_start],
        }

        self.x = x_start   # [m]
        self.y = y_start   # [m]
        self.vx = vx_start   # [m/s]
        self.vy = vy_start   # [m/s]

    def juda(self, laikas, delta_t):
        self.x += self.vx * delta_t
        self.y += self.vy * delta_t

        self.logeris['laikas'].append(laikas)
        self.logeris['x'].append(self.x)
        self.logeris['y'].append(self.y)
        self.logeris['vx'].append(self.vx)
        self.logeris['vy'].append(self.vy)


pasaulis = Pasaulis()

objektas1 = Objektas(
    x_start=0,
    y_start=0,
    vx_start=1,
    vy_start=1
)

objektas2 = Objektas(
    x_start=2,
    y_start=3,
    vx_start=1.5,
    vy_start=0
)

pasaulis.objektu_sarasas.append(objektas1)
pasaulis.objektu_sarasas.append(objektas2)

pasaulis.sukasi(
    delta_t=0.1,
    n_steps=12
)

zodynas = pasaulis.objektu_sarasas[0].logeris

image_list = []

file_list = sorted(glob.glob('*.png'))

ModuleNotFoundError: No module named 'imageio'

In [2]:
import os
import glob
import tqdm
import imageio
import numpy as np
import matplotlib.pyplot as plt

def make_movie(direktorija, movie_name, fps, delete_png):
    name_list = sorted(glob.glob(os.path.join(direktorija, '*.png')))
    image_list = [imageio.imread(name) for name in name_list]
    imageio.mimsave(
        os.path.join(direktorija, f'{movie_name}.gif'),
        image_list,
        loop=0,
        fps=fps,
    )
    if delete_png:
        for name in name_list:
            os.remove(name)

class Universe:
    def __init__(self):
        self.laikas = 0
        self.objektu_sarasas = set()

    def sudek_objektus(self, objektu_sarasas):
        self.objektu_sarasas.update(objektu_sarasas)

    def vaizduoja(self, step, direktorija):
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(9, 4))
        for objektas in self.objektu_sarasas:
            ax1.scatter(objektas.x, objektas.y)
            ax1.plot(objektas.logeris['x'][-20:], objektas.logeris['y'][-20:], lw=0.5)
        ax1.set_xlim(-200e9, 200e9)
        ax1.set_ylim(-200e9, 200e9)
        ax1.set_xlabel('$x$ [m]')
        ax1.set_ylabel('$y$ [m]')
        ax1.tick_params(direction='in', top=True, right=True)
        ax1.set_title(f'$t$ = {self.laikas:.2f} s')
        fig.savefig(os.path.join(direktorija, f'{step:03d}.png'), dpi=100)
        plt.clf()
        del fig

    def sukasi(self, delta_t, n_steps, direktorija, do_movie):
        os.makedirs(direktorija, exist_ok=True)
        for step in tqdm.tqdm(range(n_steps)):
            self.laikas += delta_t
            for objektas1 in self.objektu_sarasas:
                for objektas2 in self.objektu_sarasas:
                    if objektas1.name != objektas2.name:
                        objektas1.compute_force(objektas2)
            for objektas in self.objektu_sarasas:
                objektas.juda(self.laikas, delta_t)
            if do_movie:
                self.vaizduoja(step, direktorija)
        if do_movie:
            make_movie(direktorija, '0-movie', fps=15, delete_png=True)

class Objektas:
    """
    Represents an object in a physical system.

    Attributes:
        name (str): The name of the object.
        mass (float): The mass of the object in kilograms.
        x (float): The x-coordinate of the object's position in meters.
        y (float): The y-coordinate of the object's position in meters.
        vx (float): The x-component of the object's velocity in meters per second.
        vy (float): The y-component of the object's velocity in meters per second.
        ax (float): The x-component of the object's acceleration in meters per second squared.
        ay (float): The y-component of the object's acceleration in meters per second squared.
        force_list (list): A list to store the gravitational forces acting on the object.
        logeris (dict): A dictionary to store the object's position, velocity, acceleration, and energy over time.
    """

    def __init__(self,
                 name,
                 mass,
                 x_start, y_start,
                 vx_start, vy_start,
                 ax_start, ay_start):
        """
        Initializes a new instance of the Objektas class.

        Args:
            name (str): The name of the object.
            mass (float): The mass of the object in kilograms.
            x_start (float): The initial x-coordinate of the object's position in meters.
            y_start (float): The initial y-coordinate of the object's position in meters.
            vx_start (float): The initial x-component of the object's velocity in meters per second.
            vy_start (float): The initial y-component of the object's velocity in meters per second.
            ax_start (float): The initial x-component of the object's acceleration in meters per second squared.
            ay_start (float): The initial y-component of the object's acceleration in meters per second squared.
        """
        # Rest of the code...
class Objektas:
    """
    A class representing an object in a physical system.

    Attributes:
    - name: The name of the object.
    - mass: The mass of the object in kilograms.
    - x: The x-coordinate of the object's position in meters.
    - y: The y-coordinate of the object's position in meters.
    - vx: The x-component of the object's velocity in meters per second.
    - vy: The y-component of the object's velocity in meters per second.
    - ax: The x-component of the object's acceleration in meters per second squared.
    - ay: The y-component of the object's acceleration in meters per second squared.
    - force_list: A list of forces acting on the object.
    - logeris: A dictionary containing the object's log data.

    Methods:
    - get_energy(): Calculates the total energy of the object.
    - compute_force(other): Computes the force between the object and another object.
    - juda(laikas, delta_t): Updates the object's position and velocity based on the applied forces.
    """
    def __init__(self,
                 name,
                 mass,
                 x_start, y_start,
                 vx_start, vy_start,
                 ax_start, ay_start):
        """
        Initializes a new instance of the Objektas class.

        Parameters:
        - name: The name of the object.
        - mass: The mass of the object in kilograms.
        - x_start: The initial x-coordinate of the object's position in meters.
        - y_start: The initial y-coordinate of the object's position in meters.
        - vx_start: The initial x-component of the object's velocity in meters per second.
        - vy_start: The initial y-component of the object's velocity in meters per second.
        - ax_start: The initial x-component of the object's acceleration in meters per second squared.
        - ay_start: The initial y-component of the object's acceleration in meters per second squared.
        """
        # ... (existing code)
    
    def get_energy(self):
        """
        Calculates the total energy of the object.

        Returns:
        The total energy of the object in joules.
        """
        # ... (existing code)
    
    def compute_force(self, other):
        """
        Computes the force between the object and another object.

        Parameters:
        - other: Another instance of the Objektas class representing the other object.

        Note: The computed force is added to the force_list attribute of the object.
        """
        # ... (existing code)
    
    def juda(self, laikas, delta_t):
        """
        Updates the object's position and velocity based on the applied forces.

        Parameters:
        - laikas: The current time in seconds.
        - delta_t: The time step in seconds.

        Note: The applied forces are cleared from the force_list attribute after updating the object's state.
        """
        # ... (existing code)
class Objektas:

    def __init__(self,
                 name,
                 mass,
                 x_start, y_start,
                 vx_start, vy_start,
                 ax_start, ay_start):

        self.name = name
        self.mass = mass   # [kg]
        self.x = x_start   # [m]
        self.y = y_start   # [m]
        self.vx = vx_start   # [m/s]
        self.vy = vy_start   # [m/s]
        self.ax = ax_start   # [m/s^2]   jei noresim paskraidyt
        self.ay = ay_start   # [m/s^2]

        self.force_list = []   # kaupiame kitu objektu traukos jegas

        self.logeris = {
            'laikas': [0],   # TODO laikas 0
            'x': [x_start],
            'y': [y_start],
            'vx': [vx_start],
            'vy': [vy_start],
            'ax': [ax_start],
            'ay': [ay_start],
            'E': [self.get_energy()]
        }

    def get_energy(self):
        v = np.sqrt(self.vx**2 + self.vy**2)
        e_kin = self.mass * v**2 / 2
        # e_pot = self.mass # TODO ???? gravitacinio lauko potencine energija
        e_pot = 0   # TODO
        return e_kin + e_pot

    def compute_force(self, other):
        # kokia jega self objektas bus veikiamas other objekto
        # F = G * m1 * m2 / R^2
        # F = [Fx, Fy]
        G = 6.67e-11   # [m^3 kg^-1 s^-2]
        m1 = self.mass
        m2 = other.mass
        # vektorius (dx, dy) is self i other
        dx = other.x - self.x   # [m]
        dy = other.y - self.y
        # atstumas R
        R = np.sqrt(dx**2 + dy**2)   # [m]
        # vienetinis krypties (rx, ry) vektorius
        rx = dx / R    # [be matavimo vienetu]
        ry = dy / R
        F = G * m1 * m2 / R**2   # [N]
        # is panasiu trikampiu (nes jega veikia isilgai R)
        # fx / F = dx / R = rx
        # fy / F = dy / R = ry
        fx = rx * F
        fy = ry * F
        F = [fx, fy]
        self.force_list.append(F)

    def juda(self, laikas, delta_t):
        # suskaiciuoti jegu atstojamaja
        # ir padalinus is mases gauti pagreiti
        # self.force_list = []

        # jegu atstojamoji, kuri veikia objekta
        Fx = 0
        Fy = 0
        for F in self.force_list:
            fx, fy = F
            Fx += fx
            Fy += fy
        self.force_list = []

        self.ax = Fx / self.mass
        self.ay = Fy / self.mass

        self.vx += self.ax * delta_t
        self.vy += self.ay * delta_t
        self.x += self.vx * delta_t
        self.y += self.vy * delta_t

        self.logeris['laikas'].append(laikas)
        self.logeris['x'].append(self.x)
        self.logeris['y'].append(self.y)
        self.logeris['vx'].append(self.vx)
        self.logeris['vy'].append(self.vy)
        self.logeris['ax'].append(self.ax)
        self.logeris['ay'].append(self.ay)
        self.logeris['E'].append(self.get_energy())


universe = Universe()

obj1 = Objektas(
    name='a',
    mass=1,
    x_start=0,
    y_start=0,
    vx_start=1,
    vy_start=1,
    ax_start=0,
    ay_start=0,     # m/s / s
)

obj2 = Objektas(
    name='b',
    mass=1,
    x_start=1,
    y_start=1,
    vx_start=-1,
    vy_start=-1,
    ax_start=0,
    ay_start=0,
)

obj3 = Objektas(
    name='c',
    mass=1,
    x_start=0,
    y_start=1,
    vx_start=1,
    vy_start=-1,
    ax_start=0,
    ay_start=0,
)

# objektu_sarasas = [obj1, obj2, obj3]

obj1 = Objektas(
    name='Saule',
    mass=1.989e30,
    x_start=0,
    y_start=0,
    vx_start=0,
    vy_start=0,
    ax_start=0,
    ay_start=0,     # m/s / s
)

obj2 = Objektas(
    name='Zeme',
    mass=5.972e24,
    x_start=150e9,  # 150 mln km
    y_start=0,
    vx_start=0,
    vy_start=150e9 * 2 * 3.141 / (365.25 * 24 * 60 * 60),
    ax_start=0,
    ay_start=0,
)

objektu_sarasas = [obj1, obj2]

universe.sudek_objektus(objektu_sarasas)

universe.sukasi(
    delta_t=24 * 60 * 60,
    n_steps=365,
    direktorija='eksperimentas3',
    do_movie=True
)

# 1 paleist pav paisyma su kinematika
# 2 gravitacines jegos skaiciavimas

# pavaizduoti greicius


  5%|▌         | 20/365 [00:02<00:36,  9.44it/s]/tmp/ipykernel_11963/3353062883.py:42: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig, (ax1, ax2) = plt.subplots(
100%|██████████| 365/365 [00:42<00:00,  8.63it/s]
/tmp/ipykernel_11963/3353062883.py:15: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(name)


<Figure size 900x400 with 0 Axes>

<Figure size 900x400 with 0 Axes>

<Figure size 900x400 with 0 Axes>

<Figure size 900x400 with 0 Axes>

...

<Figure size 900x400 with 0 Axes>

<Figure size 900x400 with 0 Axes>

<Figure size 900x400 with 0 Axes>

<Figure size 900x400 with 0 Axes>

<Figure size 900x400 with 0 Axes>

In [ ]:
import os
import glob
import tqdm
import imageio
import numpy as np
import matplotlib.pyplot as plt

def make_movie(direktorija, movie_name, fps, delete_png):
    name_list = sorted(glob.glob(os.path.join(direktorija, '*.png')))
    image_list = [imageio.imread(name) for name in name_list]
    imageio.mimsave(
        os.path.join(direktorija, f'{movie_name}.gif'),
        image_list,
        loop=0,
        fps=fps,
    )
    if delete_png:
        for name in name_list:
            os.remove(name)

class Universe:
    def __init__(self):
        self.laikas = 0
        self.objektu_sarasas = set()

    def sudek_objektus(self, objektu_sarasas):
        self.objektu_sarasas.update(objektu_sarasas)

    def vaizduoja(self, step, direktorija):
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(9, 4))
        for objektas in self.objektu_sarasas:
            ax1.scatter(objektas.x, objektas.y)
            ax1.plot(objektas.logeris['x'][-20:], objektas.logeris['y'][-20:], lw=0.5)
        ax1.set_xlim(-200e9, 200e9)
        ax1.set_ylim(-200e9, 200e9)
        ax1.set_xlabel('$x$ [m]')
        ax1.set_ylabel('$y$ [m]')
        ax1.tick_params(direction='in', top=True, right=True)
        ax1.set_title(f'$t$ = {self.laikas:.2f} s')
        fig.savefig(os.path.join(direktorija, f'{step:03d}.png'), dpi=100)
        plt.clf()
        del fig

    def sukasi(self, delta_t, n_steps, direktorija, do_movie):
        os.makedirs(direktorija, exist_ok=True)
        for step in tqdm.tqdm(range(n_steps)):
            self.laikas += delta_t
            for objektas1 in self.objektu_sarasas:
                for objektas2 in self.objektu_sarasas:
                    if objektas1.name != objektas2.name:
                        objektas1.compute_force(objektas2)
            for objektas in self.objektu_sarasas:
                objektas.juda(self.laikas, delta_t)
            if do_movie:
                self.vaizduoja(step, direktorija)
        if do_movie:
            make_movie(direktorija, '0-movie', fps=15, delete_png=True)

# Rest of the code...